In [1]:
#Based off of TM-Evaluate-Models-Worksheet in that instead of getting a score for a test set we are printing bets for 
#a test set

In [2]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file

In [18]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df, get_bet_ev, get_bet_return
import random
import csv


In [4]:
%run ../automated_model_creation/MODEL_LIST.ipynb            #Load the models

In [5]:
df = pd.read_csv("../data/ufc-master.csv")
df_upcoming = pd.read_csv('../event_scraper/scraped_event.csv')

In [6]:
num_upcoming_fights = len(df_upcoming)

In [7]:
df = df_upcoming.append(df)

In [8]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [10]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [11]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[num_upcoming_fights:]
odds_train = odds_df[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = odds_df[:num_upcoming_fights]
label_test = label_df[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

10
10
10
4223
4223
4223


In [12]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
0,Amanda Nunes,Felicia Spencer,-600.0,450.0,16.666667,450.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-5.08,-2.54,-3.0,0.200000,0.000000,-0.970000,1.0,1.0,0
1,Raphael Assuncao,Cody Garbrandt,120.0,-140.0,120.000000,71.428571,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,-1.0,7.62,-2.54,-9.0,0.210000,-0.300000,-0.600000,1.0,1.0,1
2,Aljamain Sterling,Cory Sandhagen,-115.0,-105.0,86.956522,95.238095,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,-2.0,10.16,-2.54,-2.0,2.290000,-0.200000,-0.670000,1.0,1.0,0
3,Neil Magny,Anthony Rocco Martin,-135.0,115.0,74.074074,115.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Red,...,2.0,-7.62,-17.78,-2.0,-1.230000,0.800000,-2.020000,1.0,1.0,0
4,Eddie Wineland,Sean O'Malley,400.0,-500.0,400.000000,20.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,10.16,7.62,-10.0,3.460000,1.100000,0.960000,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,Duane Ludwig,Darren Elkins,-155.0,135.0,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0.0,0.00,2.54,6.0,-13.666667,0.000000,0.000000,0.0,1.0,1
4292,John Howard,Daniel Roberts,-210.0,175.0,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,7.62,7.62,-2.0,-18.000000,-1.000000,-4.666667,0.0,1.0,0
4293,Brendan Schaub,Chase Gormley,-260.0,220.0,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,-2.54,-2.12,0.0,-4.000000,1.000000,1.000000,0.0,1.0,0
4294,Mike Pierce,Julio Paulino,-420.0,335.0,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,10.16,7.62,-5.0,-40.500000,0.000000,-3.500000,0.0,1.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
0,Jessica Eye,Cynthia Calvillo,120.0,-130.0,120.000000,76.923077,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,3.0,-5.08,-5.08,-1.0,0.33,0.9,1.37,1.0,1.0,0
1,Karl Roberson,Marvin Vettori,210.0,-230.0,210.000000,43.478261,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-2.54,0.00,-3.0,0.89,-0.2,0.36,1.0,1.0,0
2,Charles Rosa,Kevin Aguilar,170.0,-185.0,170.000000,54.054054,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,-2.0,-5.08,10.16,-2.0,0.52,-1.0,-2.11,1.0,1.0,0
3,Andre Fili,Charles Jourdain,-220.0,200.0,45.454545,200.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,0.0,-5.08,-12.70,-5.0,0.85,-0.2,-2.49,1.0,1.0,0
4,Jordan Espinosa,Mark De La Rosa,-167.0,157.0,59.880240,157.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,1.0,0.00,-10.16,-5.0,-0.97,-0.2,-0.19,1.0,1.0,0
5,Mariya Agapova,Hannah Cifers,-335.0,305.0,29.850746,305.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,0.0,-12.70,-15.24,4.0,2.11,0.3,0.31,1.0,1.0,0
6,Merab Dvalishvili,Gustavo Lopez,-800.0,635.0,12.500000,635.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,0.0,-2.54,-2.54,1.0,-3.84,0.0,-7.80,1.0,1.0,0
7,Julia Avila,Gina Mazany,-575.0,485.0,17.391304,485.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,0.0,-2.54,0.00,-1.0,-0.55,0.0,3.18,1.0,1.0,0
8,Tyson Nam,Zarrukh Adashev,-135.0,125.0,74.074074,125.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,0.0,-5.08,-7.62,-9.0,-3.00,0.0,0.00,1.0,1.0,0
9,Anthony Ivy,Christian Aguilera,-185.0,170.0,54.054054,170.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,0.0,-12.70,-5.08,-2.0,0.00,0.0,0.00,1.0,1.0,0


In [25]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        elif t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        else:
            print(f"No bets.  EV1:{t1_bet_ev}. EV2: {t2_bet_ev}")
        
        print()
        
    return(0)
        

In [14]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    #Train model.  
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]

    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #display(fighters_test)
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, -2], odds_test[:, -1], 
                            probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name', 't1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    #display(ev_prepped_df)
    
    display_bets(ev_prepped_df, input_ev)
    
    return(0)

In [22]:
with open('../automated_model_creation/models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

print(models)

model = eval(models[1][0])
features = eval(models[2][0])
ev = eval(models[3][0])

print(model, features, ev)

[['model_1', 'model_2', 'model_3', 'model_4', 'model_5', ''], ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)", "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=262,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')", "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='gini', max_depth=None, max_features='auto',\n       

In [26]:

get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)


Jessica Eye vs. Cynthia Calvillo
Jessica Eye has an EV of 10.25 on odds of 120.0.  They have 50.11 % chance of winning.

Karl Roberson vs. Marvin Vettori
Karl Roberson has an EV of 13.24 on odds of 210.0.  They have 36.53 % chance of winning.

Charles Rosa vs. Kevin Aguilar
Charles Rosa has an EV of 8.48 on odds of 170.0.  They have 40.18 % chance of winning.

Andre Fili vs. Charles Jourdain
No bets.  EV1:-2.3689218261662184. EV2: -1.364098733532174

Jordan Espinosa vs. Mark De La Rosa
No bets.  EV1:-1.8957583200647719. EV2: -0.6979756052861532

Mariya Agapova vs. Hannah Cifers
Mariya Agapova has an EV of 2.27 on odds of -335.0.  They have 78.76 % chance of winning.

Merab Dvalishvili vs. Gustavo Lopez
Merab Dvalishvili has an EV of 5.81 on odds of -800.0.  They have 94.06 % chance of winning.

Julia Avila vs. Gina Mazany
Julia Avila has an EV of 5.06 on odds of -575.0.  They have 89.49 % chance of winning.

Tyson Nam vs. Zarrukh Adashev
No bets.  EV1:-2.4713710301162095. EV2: -1.06094

0